In [2]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
from common import sanitize
# import torch.nn as nn
# torch.nn.modules.module.Module._call_impl = _call_impl2
# torch.nn.Module._call_impl=_call_impl2
# nn.Module._call_impl=_call_impl2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

import time
from transformers import AutoModel
from transformers import AutoConfig
import torch
from torch import nn
torch.set_grad_enabled(False)

In [85]:
from calibrate_e_ml import get_module_info
from calibrate_e_ml import print_info

from cg.node import construct_aggregation_graph

In [4]:
seq_len = 100
bs = 4
input_ids = torch.randint(1000, size=(bs, seq_len), dtype=torch.long, device='cpu')
cuda_exist = torch.cuda.is_available()
device = torch.device("cuda" if cuda_exist else "cpu")

In [5]:
model_name = "prajjwal1/bert-tiny"
information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(information)


BertEmbeddings 1
<class 'dict'> embeddings:BertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

BertSelfAttention 2
<class 'tuple'> encoder.layer.0.attention.self:BertSelfAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

BertSelfOutput 2
<class 'tuple'> encoder.layer.0.attention.output:BertSelfOutput [torch.Size([4, 100, 128]), torch.Size([4, 100, 128])]

BertAttention 2
<class 'dict'> encoder.layer.0.attention:BertAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:BertAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None]

BertIntermediate 2
<class 'tuple'> encoder.layer.0.intermediate:BertIntermediate [torch.Size([4, 100, 128])]

BertOutput 2
<class 'tuple'> encoder.layer.0.output:BertOutput [torch.Size([4, 100, 512]), torch.Size([4, 100, 128])]

BertLaye

In [6]:
import inspect
from run_level_exp import get_model_flops_mem_bytes, wrapped_partial

In [7]:
sel_attn = information['BertSelfAttention'][0]
fn = sel_attn['module']
fi = sel_attn['inputs']
fi_kwargs = sel_attn['in_kwargs']

fn_fwd = fn.forward
#  unify fi and fi_kwargs
fn_args = inspect.getfullargspec(fn_fwd).args
fill_args = dict()
fi_args = fn_args[1:1 + len(fi)]
ti = []
for fi_k, fi_v in zip(fi_args, fi):
    if isinstance(fi_v, torch.Tensor):
        ti.append(fi_v)
    else:
        fill_args[fi_k] = fi_v
for k, v in fi_kwargs.items():
    if isinstance(v, torch.Tensor):
        ti.append(v)
    else:
        fill_args[k] = v
# wrap forward into traceable fn (only tensor args)
# https://github.com/pytorch/pytorch/issues/14455#issuecomment-445962680
fn.forward = wrapped_partial(fn.forward, **fill_args)

In [8]:
flops, mem_bytes = get_model_flops_mem_bytes(fn, ti, 'BertEncoder')
print(flops/1e6, mem_bytes/1024/1024)

60.5152 6.62158203125


In [27]:
model_name = "distilbert-base-uncased"
distilbert_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(distilbert_information)

Embeddings 1
<class 'tuple'> embeddings:Embeddings [torch.Size([4, 100])]

MultiHeadSelfAttention 6
<class 'dict'> transformer.layer.0.attention:MultiHeadSelfAttention {'query': torch.Size([4, 100, 768]), 'key': torch.Size([4, 100, 768]), 'value': torch.Size([4, 100, 768]), 'mask': torch.Size([4, 100]), 'head_mask': None, 'output_attentions': False}

FFN 6
<class 'tuple'> transformer.layer.0.ffn:FFN [torch.Size([4, 100, 768])]

TransformerBlock 6
<class 'dict'> transformer.layer.0:TransformerBlock {'x': torch.Size([4, 100, 768]), 'attn_mask': torch.Size([4, 100]), 'head_mask': None, 'output_attentions': False}

Transformer 1
<class 'dict'> transformer:Transformer {'x': torch.Size([4, 100, 768]), 'attn_mask': torch.Size([4, 100]), 'head_mask': [None, None, None, None, None, None], 'output_attentions': False, 'output_hidden_states': False, 'return_dict': False}



In [42]:
get_non_parametric_ml_op(distilbert_information)

Embeddings,embeddings:Embeddings

MultiHeadSelfAttention,transformer.layer.0.attention:MultiHeadSelfAttention
		OpNode(id=scores, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=q, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=53, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=scores, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.3, op=aten::softmax, flops=2400000, mem_bytes=5760000, 
			inputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=64, dtype=int)
				DataNode(id=65, dtype=None)],
			outputs=[
				DataNode(id=input.3, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=x, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=weights, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=v, dtype=Float, shape=[4, 12, 100, 64])],
			outputs=[
				DataNode(id=x, dtype=Float, shape=[4, 12, 100, 64])],)

MultiHeadSelfAttention,transformer.layer.1.at

/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/modeling_utils.py:1759: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert all(



FFN,transformer.layer.1.ffn:FFN

FFN,transformer.layer.2.ffn:FFN

FFN,transformer.layer.3.ffn:FFN

FFN,transformer.layer.4.ffn:FFN

FFN,transformer.layer.5.ffn:FFN

TransformerBlock,transformer.layer.0:TransformerBlock
		OpNode(id=input.2, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=attention, 
			inputs=[
				DataNode(id=input.1, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=26, dtype=int)
				DataNode(id=19, dtype=None)],
			outputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])],)

TransformerBlock,transformer.layer.1:TransformerBlock
		OpNode(id=input.2, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=attention, 
			inputs=[
				DataNode(id=input.1, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=26, dtype=int)
				DataNode(id=19, dtype=None)],
			outputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])],)

TransformerBlock,transformer.layer.2:TransformerBlock
		OpNode(id=input.2, op=aten::softmax, flops=24000

In [26]:
model_name = "bert-base-uncased"
bert_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(bert_information)

BertEmbeddings 1
<class 'dict'> embeddings:BertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

BertSelfAttention 12
<class 'tuple'> encoder.layer.0.attention.self:BertSelfAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

BertSelfOutput 12
<class 'tuple'> encoder.layer.0.attention.output:BertSelfOutput [torch.Size([4, 100, 768]), torch.Size([4, 100, 768])]

BertAttention 12
<class 'dict'> encoder.layer.0.attention:BertAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:BertAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None]

BertIntermediate 12
<class 'tuple'> encoder.layer.0.intermediate:BertIntermediate [torch.Size([4, 100, 768])]

BertOutput 12
<class 'tuple'> encoder.layer.0.output:BertOutput [torch.Size([4, 100, 3072]), torch.Size([4, 100, 768])]

Ber

In [36]:
get_non_parametric_ml_op(bert_information['BertSelfAttention'])

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=query_layer, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=66, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input, op=aten::softmax, flops=2400000, mem_bytes=5760000, 
			inputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=72, dtype=int)
				DataNode(id=73, dtype=None)],
			outputs=[
				DataNode(id=input, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=context_layer.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=attention_probs, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=value_layer, dtype=Float, shape=[4, 12, 100, 64])],
			outputs=[
				DataNode(id=context_layer.1, dtype=Float, shape=[4, 12, 100, 64])],)


In [25]:
model_name = "roberta-base"
roberta_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(roberta_information)

RobertaEmbeddings 1
<class 'dict'> embeddings:RobertaEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

RobertaSelfAttention 12
<class 'tuple'> encoder.layer.0.attention.self:RobertaSelfAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

RobertaSelfOutput 12
<class 'tuple'> encoder.layer.0.attention.output:RobertaSelfOutput [torch.Size([4, 100, 768]), torch.Size([4, 100, 768])]

RobertaAttention 12
<class 'dict'> encoder.layer.0.attention:RobertaAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:RobertaAttention [torch.Size([4, 100, 768]), torch.Size([4, 1, 1, 100]), None]

RobertaIntermediate 12
<class 'tuple'> encoder.layer.0.intermediate:RobertaIntermediate [torch.Size([4, 100, 768])]

RobertaOutput 12
<class 'tuple'> encoder.layer.0.output:RobertaOutput [torch.Size([4, 100, 

In [31]:
get_non_parametric_ml_op(roberta_information['RobertaSelfAttention'])

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=query_layer, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=66, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input, op=aten::softmax, flops=2400000, mem_bytes=5760000, 
			inputs=[
				DataNode(id=input.2, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=72, dtype=int)
				DataNode(id=73, dtype=None)],
			outputs=[
				DataNode(id=input, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=context_layer.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, 
			inputs=[
				DataNode(id=attention_probs, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=value_layer, dtype=Float, shape=[4, 12, 100, 64])],
			outputs=[
				DataNode(id=context_layer.1, dtype=Float, shape=[4, 12, 100, 64])],)


In [24]:
# for k, info in information.items():
#     for infoi in info:
#         ii = infoi['inputs']
#         if isinstance(ii, dict):
#             print(type(ii), infoi['name'],  {iik:v.shape if isinstance(v, torch.Tensor) else v for iik, v in ii.items()})
#         elif isinstance(ii, tuple):
#             print(type(ii), infoi['name'],  [v.shape if isinstance(v, torch.Tensor) else v for v in ii])
#         else:
#             print(type(ii), 'todo', k, infoi)
#         print()
model_name = "google/mobilebert-uncased"
mobilebert_information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(mobilebert_information)

NoNorm 193
<class 'tuple'> embeddings.LayerNorm:NoNorm [torch.Size([4, 100, 512])]

MobileBertEmbeddings 1
<class 'dict'> embeddings:MobileBertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None}

BottleneckLayer 48
<class 'tuple'> encoder.layer.0.bottleneck.input:BottleneckLayer [torch.Size([4, 100, 512])]

Bottleneck 24
<class 'tuple'> encoder.layer.0.bottleneck:Bottleneck [torch.Size([4, 100, 512])]

MobileBertSelfAttention 24
<class 'tuple'> encoder.layer.0.attention.self:MobileBertSelfAttention [torch.Size([4, 100, 128]), torch.Size([4, 100, 128]), torch.Size([4, 100, 512]), torch.Size([4, 1, 1, 100]), None, False]

MobileBertSelfOutput 24
<class 'tuple'> encoder.layer.0.attention.output:MobileBertSelfOutput [torch.Size([4, 100, 128]), torch.Size([4, 100, 128])]

MobileBertAttention 24
<class 'dict'> encoder.layer.0.attention:MobileBertAttention {'output_attentions': False}
<class 'tuple'> encoder.layer

In [35]:
get_non_parametric_ml_op(mobilebert_information['MobileBertSelfAttention'])

		OpNode(id=attention_scores.1, op=aten::matmul, flops=10240000, mem_bytes=1049600, 
			inputs=[
				DataNode(id=query_layer, dtype=Float, shape=[4, 4, 100, 32])
				DataNode(id=68, dtype=Float, shape=[4, 4, 32, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 4, 100, 100])],)
		OpNode(id=input, op=aten::softmax, flops=800000, mem_bytes=1920000, 
			inputs=[
				DataNode(id=input.4, dtype=Float, shape=[4, 4, 100, 100])
				DataNode(id=74, dtype=int)
				DataNode(id=75, dtype=None)],
			outputs=[
				DataNode(id=input, dtype=Float, shape=[4, 4, 100, 100])],)
		OpNode(id=context_layer.1, op=aten::matmul, flops=10240000, mem_bytes=1049600, 
			inputs=[
				DataNode(id=attention_probs, dtype=Float, shape=[4, 4, 100, 100])
				DataNode(id=value_layer, dtype=Float, shape=[4, 4, 100, 32])],
			outputs=[
				DataNode(id=context_layer.1, dtype=Float, shape=[4, 4, 100, 32])],)


In [13]:
model_name = "prajjwal1/bert-tiny"
information = get_module_info(model_name, bs, seq_len, device, 'module')
print_info(information)

BertEmbeddings 1
<class 'dict'> embeddings:BertEmbeddings {'input_ids': torch.Size([4, 100]), 'position_ids': None, 'token_type_ids': torch.Size([4, 100]), 'inputs_embeds': None, 'past_key_values_length': 0}

BertSelfAttention 2
<class 'tuple'> encoder.layer.0.attention.self:BertSelfAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None, None, None, None, False]

BertSelfOutput 2
<class 'tuple'> encoder.layer.0.attention.output:BertSelfOutput [torch.Size([4, 100, 128]), torch.Size([4, 100, 128])]

BertAttention 2
<class 'dict'> encoder.layer.0.attention:BertAttention {'output_attentions': False, 'past_key_value': None}
<class 'tuple'> encoder.layer.0.attention:BertAttention [torch.Size([4, 100, 128]), torch.Size([4, 1, 1, 100]), None]

BertIntermediate 2
<class 'tuple'> encoder.layer.0.intermediate:BertIntermediate [torch.Size([4, 100, 128])]

BertOutput 2
<class 'tuple'> encoder.layer.0.output:BertOutput [torch.Size([4, 100, 512]), torch.Size([4, 100, 128])]

BertLayer

In [46]:
import torch
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from transformers import modeling_utils

def is_ml_operation(module):
    """
    This function checks if any given module is of a type that
    we want to analyse for E_ML operations
    """

    e_ml_operations = {nn.Linear, nn.LayerNorm, nn.Embedding, nn.BatchNorm1d,
                       nn.Conv1d, nn.MaxPool1d, nn.AvgPool1d, nn.LSTM, nn.Tanh,
                       modeling_utils.Conv1D}

    for e_ml_op in e_ml_operations:
        if isinstance(module, e_ml_op):
            return True
    return False

config = AutoConfig.from_pretrained(model_name)
config.torchscript = True
model=AutoModel.from_config(config).eval()

In [94]:
model.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torchscript": true,
  "transformers_version": "4.2.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [16]:
for name, module in model.named_modules():
    if not name:
        continue
    if not is_ml_operation(module):
        print(f'{name}:{module.__class__.__name__}')
#         print(f'\t{module}\n')

embeddings:BertEmbeddings
embeddings.dropout:Dropout
encoder:BertEncoder
encoder.layer:ModuleList
encoder.layer.0:BertLayer
encoder.layer.0.attention:BertAttention
encoder.layer.0.attention.self:BertSelfAttention
encoder.layer.0.attention.self.dropout:Dropout
encoder.layer.0.attention.output:BertSelfOutput
encoder.layer.0.attention.output.dropout:Dropout
encoder.layer.0.intermediate:BertIntermediate
encoder.layer.0.output:BertOutput
encoder.layer.0.output.dropout:Dropout
encoder.layer.1:BertLayer
encoder.layer.1.attention:BertAttention
encoder.layer.1.attention.self:BertSelfAttention
encoder.layer.1.attention.self.dropout:Dropout
encoder.layer.1.attention.output:BertSelfOutput
encoder.layer.1.attention.output.dropout:Dropout
encoder.layer.1.intermediate:BertIntermediate
encoder.layer.1.output:BertOutput
encoder.layer.1.output.dropout:Dropout
pooler:BertPooler


In [88]:

def get_non_parametric_ml_ops(model, input_ids):
    non_param_scopes = {name for name, module in model.named_modules() if not is_ml_operation(module)}
    trace = torch.jit.trace(model, input_ids)
    trace_graph = trace.inlined_graph
    graph, op_data_types = construct_aggregation_graph(trace_graph, fname)
    sigs = set()
    for node in graph.nodes:
        if node.op == 'aten::softmax':
            sig = f'op=softmax,flops={node.flops},mem_bytes={node.mem_bytes}'
            if sig not in sigs:
                print(node)
                sigs.add(sig)
        if node.op == 'aten::matmul' and node.scope in non_param_scopes:
            sig = f'op=matmul,flops={node.flops},mem_bytes={node.mem_bytes}'
            if sig not in sigs:
                print(node)
                sigs.add(sig)

In [102]:
from cg.node import construct_aggregation_graph
from calibrate_e_ml import get_non_parametric_ml_ops
model_name = "prajjwal1/bert-tiny"
# "google/mobilebert-uncased" "bert-base-uncased" "distilbert-base-uncased" "roberta-base" "prajjwal1/bert-tiny"
config = AutoConfig.from_pretrained(model_name)
config.torchscript = True
model=AutoModel.from_config(config).eval()
np_info = get_non_parametric_ml_ops(model, input_ids)

In [103]:
print_info(np_info)

matmul 4
<class 'tuple'> encoder.layer.0.attention.self [torch.Size([4, 2, 100, 64]), torch.Size([4, 2, 64, 100])]

softmax 2
<class 'tuple'> encoder.layer.0.attention.self [torch.Size([4, 2, 100, 100])]



In [83]:
for node in trace_graph.nodes():
    if node.kind() == 'prim::Constant':
#         print(node)
        print(node.outputs().toIValue())

0
1
tensor(0)
1
6
None
cpu
False
4
None
cpu
False
0
0
9223372036854775807
1
1
2
3
0
9223372036854775807
1
6
False
False
None
1.0
1
tensor(-10000., dtype=torch.float64)
0.1
128
1e-12
True
-1
False
9223372036854775807
0
tensor(0)
1
True
1e-12
1
0
2
64
3
-1
-2
tensor(8., dtype=torch.float64)
None
False
0.1
128
1
9223372036854775807
0


In [104]:
config = AutoConfig.from_pretrained("google/mobilebert-uncased")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/models/mobilebert/modeling_mobilebert.py:192: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  position_ids = self.position_ids[:, :seq_length]
/Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/transformers/models/mobilebert/modeling_mobilebert.py:526: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(1000),


defaultdict(list,
            {'matmul': [{'name': 'encoder.layer.0.attention.self',
               'module': MatMul(),
               'inputs': (tensor([[[[0.1530, 0.5040, 0.9343,  ..., 0.2472, 0.0243, 0.9528],
                          [0.0908, 0.1480, 0.7900,  ..., 0.2535, 0.3581, 0.3396],
                          [0.3862, 0.8281, 0.3161,  ..., 0.0687, 0.7563, 0.7296],
                          ...,
                          [0.7469, 0.4792, 0.9273,  ..., 0.3584, 0.9260, 0.5957],
                          [0.5594, 0.4355, 0.3560,  ..., 0.6293, 0.7538, 0.3437],
                          [0.9275, 0.1655, 0.2467,  ..., 0.6217, 0.6327, 0.3315]],
                
                         [[0.3078, 0.5492, 0.8378,  ..., 0.5599, 0.3487, 0.5249],
                          [0.4503, 0.6921, 0.5641,  ..., 0.4652, 0.7415, 0.1776],
                          [0.5181, 0.0584, 0.2797,  ..., 0.8831, 0.1082, 0.0707],
                          ...,
                          [0.1651, 0.0886, 0.4231,  

In [67]:
config = AutoConfig.from_pretrained("bert-base-uncased")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=query_layer.1, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=172, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.7, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=input.6, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=100, dtype=int)
				DataNode(id=103, dtype=None)],
			outputs=[
				DataNode(id=input.7, dtype=Float, shape=[4, 12, 100, 100])],)


In [68]:
config = AutoConfig.from_pretrained("distilbert-base-uncased")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

		OpNode(id=scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, scope=transformer.layer.0.attention, 
			inputs=[
				DataNode(id=q.2, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=110, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.5, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=transformer.layer.0.attention, 
			inputs=[
				DataNode(id=input.4, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=58, dtype=int)
				DataNode(id=51, dtype=None)],
			outputs=[
				DataNode(id=input.5, dtype=Float, shape=[4, 12, 100, 100])],)


In [69]:
config = AutoConfig.from_pretrained("roberta-base")
config.torchscript = True
model=AutoModel.from_config(config).eval()
get_non_parametric_ml_ops(model, input_ids)

		OpNode(id=attention_scores.1, op=aten::matmul, flops=61440000, mem_bytes=4377600, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=query_layer.1, dtype=Float, shape=[4, 12, 100, 64])
				DataNode(id=178, dtype=Float, shape=[4, 12, 64, 100])],
			outputs=[
				DataNode(id=attention_scores.1, dtype=Float, shape=[4, 12, 100, 100])],)
		OpNode(id=input.7, op=aten::softmax, flops=2400000, mem_bytes=5760000, scope=encoder.layer.0.attention.self, 
			inputs=[
				DataNode(id=input.6, dtype=Float, shape=[4, 12, 100, 100])
				DataNode(id=106, dtype=int)
				DataNode(id=109, dtype=None)],
			outputs=[
				DataNode(id=input.7, dtype=Float, shape=[4, 12, 100, 100])],)


In [1]:
import bisect
import json
import pandas as pd
import numpy as np
from pathlib import Path
from common import sanitize

pd.set_option('display.float_format', '{:.6f}'.format)
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [41]:
res_names = ['cpu', 'mem', 'gpu', 'gpu_mem']

feature_names = ['batch_size', 'seq_len', 'flops',
                 'mem_bytes'] + \
                ['times_mean', 'times_std',
                 'gpu_power_mean', 'gpu_power_std', 'repeats',
                 'energy_mean', 'energy_std',
                 'level_name', 'model_name'] + res_names + [f'{k}_std' for k in res_names] 

def process_record(energy, prof_info, res, feature_values,
                   model_name, bs, runs, seq_len):
    res_np = res.to_numpy()
    res_t = res_np[:, 0]
    energy_np = energy.to_numpy()
    energy_t = energy_np[:, 0]
    for prof_item in prof_info:
        gpu_power_runs = []
        energy_runs = []
        times_runs = []
        res_runs = {k: [] for k in res_names}
        repeats = prof_item['repeats']
        for r in range(1, runs + 1):
            start_r = prof_item[f'start_{r}']
            end_r = prof_item[f'end_{r}']
            times_runs.append(end_r - start_r)

            res_s = bisect.bisect_right(res_t, start_r)
            res_e = bisect.bisect_right(res_t, end_r)
            res_r = res[res_s:res_e]
            for rn in res_names:
                res_runs[rn].append(res_r[rn].mean())
            gpu_power_r = res_r['gpu_power'].sum()
            gpu_power_runs.append(gpu_power_r)

            e_s = bisect.bisect_right(energy_t, start_r)
            e_e = bisect.bisect_right(energy_t, end_r)
            energy_r = energy[e_s:e_e]['value'].div(repeats).sum()
            energy_runs.append(energy_r)

        times_mean = np.mean(times_runs)
        times_std = np.std(times_runs) / times_mean * 100
        gpu_power_mean = np.mean(gpu_power_runs)
        gpu_power_std = np.std(gpu_power_runs) / gpu_power_mean * 100
        energy_mean = np.median(energy_runs)
        energy_std = np.std(energy_runs) / energy_mean * 100
        for rn in res_names:
            feature_values[rn].append(np.mean(res_runs[rn]))
            rn_std = np.std(res_runs[rn]) / np.mean(res_runs[rn]) * 100
            feature_values[f'{rn}_std'].append(rn_std)

        flops = prof_item['flops'] / 1e6
        mem_bytes = prof_item['mem_bytes'] / 1024 / 1024
        feature_values['batch_size'].append(bs)
        feature_values['seq_len'].append(seq_len)
        feature_values['repeats'].append(repeats)
        feature_values['energy_mean'].append(energy_mean)
        feature_values['energy_std'].append(energy_std)
        feature_values['gpu_power_mean'].append(gpu_power_mean)
        feature_values['gpu_power_std'].append(gpu_power_std)
        feature_values['flops'].append(flops)
        feature_values['mem_bytes'].append(mem_bytes)
        feature_values['times_mean'].append(times_mean)
        feature_values['times_std'].append(times_std)
        feature_values['level_name'].append(prof_item['name'])
        feature_values['model_name'].append(model_name)


In [46]:
args=Namespace(out_dir='data/jpc', 
               models=["bert-base-uncased", "distilbert-base-uncased", "roberta-base"],
               exp_type='module', exp_name='module-exp-jpc', runs=5, 
               input_start=240, seq_step=8, input_length=264,
               batch_start=24, batch_step=8, batch_size=40, 
               )

out_dir = Path(args.out_dir)
out_dir.mkdir(parents=True, exist_ok=True)
runs = args.runs
input_start = args.input_start
input_length = args.input_length
batch_start = args.batch_start
batch_size = args.batch_size
batch_step = args.batch_step
seq_step = args.seq_step
exp_type = args.exp_type
exp_name = args.exp_name

feature_file = out_dir / f'{exp_name}_features.csv'
res_file =  out_dir / f'three-exp-jpc-res.csv'
energy_file = out_dir / f'three-exp-jpc-energy.csv'

In [47]:

res = pd.read_csv(res_file).dropna()
res = res.apply(pd.to_numeric)

energy = pd.read_csv(energy_file, error_bad_lines=False, usecols=[0, 2]).dropna()
energy = energy[energy['value'].apply(lambda x: is_float(x))]
energy = energy[energy['timestamp'].apply(lambda x: is_float(x))]

energy['value'] = energy['value'].astype(float).div(100)
energy['timestamp'] = energy['timestamp'].astype(float)


In [44]:
infos = []
for model_name in args.models:
    feature_values = {k: [] for k in feature_names}
    for bs in range(batch_start, batch_size, batch_step):
        for seq_len in range(input_start, input_length, seq_step):
            name_s = sanitize(model_name)
            filename = f'{name_s}_{exp_type}_r{runs}_b{bs}_i{seq_len}.json'
            prof_file = Path(out_dir) / exp_name / filename
            if not prof_file.exists():
                continue
            with open(prof_file) as f:
                prof_info = json.load(f)
            process_record(energy, prof_info, res, feature_values,
                           model_name, bs, runs, seq_len)
    info = pd.DataFrame(data=feature_values)
    info.to_csv(feature_file)
    infos.append(info)
    print(f'{model_name} done.')

bert-base-uncased done.
distilbert-base-uncased done.
roberta-base done.


In [50]:
bs = 32
seq_len = 256
runs = 3
model_name = "bert-base-uncased"

feature_values = {k: [] for k in feature_names}
name_s = sanitize(model_name)
filename = f'{name_s}_{exp_type}_r{runs}_b{bs}_i{seq_len}.json'
prof_file = Path(out_dir) / exp_name / filename
with open(prof_file) as f:
    prof_info = json.load(f)
process_record(energy, prof_info, res, feature_values, model_name, bs, runs, seq_len)
info = pd.DataFrame(data=feature_values)

In [51]:
pd.set_option('display.float_format', '{:.6f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [52]:
info

,batch_size,seq_len,flops,mem_bytes,times_mean,times_std,gpu_power_mean,gpu_power_std,repeats,energy_mean,energy_std,level_name,model_name,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std
0,32,256,62.914560,240.876953,64.190070,0.430990,74290093.666667,1.805988,57000,3.899590,5.300172,embeddings:BertEmbeddings,bert-base-uncased,5.699669,5.000000,99.836010,70.927237,0.674398,0.000000,0.117064,0.638445
1,32,256,35628.515328,1464.000000,67.178517,0.392687,79198218.666667,0.349591,8000,30.225859,0.318522,encoder.layer.0.attention.self:BertSelfAttention,bert-base-uncased,5.705381,5.000000,99.768190,49.883249,0.199312,0.000000,0.176463,0.176841
2,32,256,9726.590976,312.000000,67.996897,0.175519,81309436.333333,0.408668,38000,6.486546,1.762590,encoder.layer.0.attention.output:BertSelfOutput,bert-base-uncased,5.322210,12.000000,99.865352,43.943080,1.087201,0.000000,0.098165,0.152783
3,32,256,45355.106304,1776.000000,71.326757,0.393425,84318578.666667,0.208027,7000,36.044283,1.975930,encoder.layer.0.attention:BertAttention,bert-base-uncased,5.474086,16.000000,99.813946,48.836662,0.399415,0.000000,0.134487,0.064783
4,32,256,38830.866432,792.000000,82.299969,0.563966,109761647.666667,0.750342,14500,1.094620,4.886139,encoder.layer.0.intermediate:BertIntermediate,bert-base-uncased,5.214368,20.000000,100.000000,44.006845,0.413468,0.000000,0.000000,0.021996
5,32,256,38717.620224,600.000000,73.608322,0.452312,95851639.333333,0.766047,14500,0.732261,3.254426,encoder.layer.0.output:BertOutput,bert-base-uncased,5.434978,24.000000,100.000000,26.760684,0.300812,0.000000,0.000000,0.310718
6,32,256,122903.592960,3168.000000,73.049241,0.375926,91655737.333333,0.576630,3500,1.260026,74.271545,encoder.layer.0:BertLayer,bert-base-uncased,5.566868,28.000000,99.866627,42.021367,0.761294,0.000000,0.103075,0.243554
7,32,256,1474843.115520,38016.000000,125.833236,0.065132,157947671.666667,0.756771,500,21.259080,3.803905,encoder:BertEncoder,bert-base-uncased,5.563415,39.000000,99.969286,42.085656,0.194839,0.000000,0.021753,0.162816
8,32,256,37.847040,2.531250,60.183214,0.200130,42377302.666667,1.156435,1207500,0.133658,0.227659,pooler:BertPooler,bert-base-uncased,5.201026,39.000000,47.450816,3.148473,0.224442,0.000000,0.444754,3.317750


In [12]:
res.to_csv('data/jpc/res.csv')

In [137]:
for info in infos:
    print(info.shape)

(8, 20)
(5, 20)
(2, 20)


In [30]:
ml_np_info = pd.concat(infos)
ml_np_info.dropna()
# ml_np_info.dropna().to_csv('data/jpc/ml_np_features.csv')

,batch_size,seq_len,flops,mem_bytes,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std
0,24,256,2415.919104,108.000000,5.147783,3.286096,6795454.200000,2.952834,0.081395,23.258022,encoder.layer.0.attention.self,bert-base-uncased,5.235148,5.000000,97.077701,76.655086,3.299899,0.000000,1.722676,1.706944
1,24,256,94.371840,216.000000,5.103131,3.395346,6240633.800000,2.728518,0.971917,19.610943,encoder.layer.0.attention.self,bert-base-uncased,5.644300,5.000000,97.340230,97.278621,1.276814,0.000000,1.391051,1.420210
2,32,256,3221.225472,144.000000,5.187629,4.048708,7572261.200000,2.741838,0.229181,8.219304,encoder.layer.0.attention.self,bert-base-uncased,5.412958,4.000000,97.093333,78.537083,1.210142,0.000000,1.711115,1.843821
3,32,256,125.829120,288.000000,5.163217,4.503849,7399339.200000,5.045464,0.234894,4.490870,encoder.layer.0.attention.self,bert-base-uncased,5.166825,6.000000,97.474253,97.487586,1.157011,0.000000,1.406107,1.397432
0,24,256,2415.919104,108.000000,5.133333,3.270081,6771262.400000,3.342704,0.090859,17.796357,transformer.layer.0.attention,distilbert-base-uncased,5.146408,5.000000,97.497011,77.021839,2.658152,0.000000,1.878248,1.876260
2,32,256,3221.225472,144.000000,5.186796,4.316623,6887177.400000,4.107880,0.134683,15.436467,transformer.layer.0.attention,distilbert-base-uncased,5.177375,4.000000,97.030575,76.675402,1.258968,0.000000,1.796582,1.773788
3,32,256,125.829120,288.000000,5.151971,4.420412,6305656.800000,1.706890,1.049405,35.907798,transformer.layer.0.attention,distilbert-base-uncased,5.642159,5.000000,97.498621,97.594253,1.783116,0.000000,1.903388,1.905324
0,32,256,3221.225472,144.000000,5.147739,4.236269,7019361.000000,5.996940,0.145029,15.273480,encoder.layer.0.attention.self,roberta-base,5.247576,4.000000,97.076552,77.612665,3.392524,0.000000,1.513285,1.612249
1,32,256,125.829120,288.000000,5.136242,4.502653,6146730.400000,4.827077,1.262912,9.665152,encoder.layer.0.attention.self,roberta-base,5.700934,6.000000,97.590115,97.597931,1.530835,0.000000,1.855638,1.916548


In [23]:
ml_info = pd.concat(infos)
ml_info

,batch_size,seq_len,flops,mem_bytes,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std
0,24,256,0.000000,18.046875,5.297559,0.671366,6487430.600000,6.064336,0.360137,41.511543,embeddings.word_embeddings:Embedding,bert-base-uncased,5.554093,5.000000,96.744960,49.171976,0.863226,0.000000,1.889926,3.435478
1,24,256,0.000000,0.751953,4.986228,0.375347,4374928.000000,1.052883,0.047629,2.670731,embeddings.position_embeddings:Embedding,bert-base-uncased,5.121011,5.000000,45.393793,10.606897,0.802889,0.000000,2.083743,2.113995
2,24,256,37.748736,72.000000,5.225760,1.223196,6406707.400000,6.645233,0.481666,43.645322,embeddings.LayerNorm:LayerNorm,bert-base-uncased,5.547828,5.000000,96.793548,67.973763,2.340220,0.000000,2.074793,3.905903
3,24,256,7252.475904,126.000000,5.265767,3.863429,6720833.200000,3.438980,0.160668,25.029862,encoder.layer.0.attention.self.query:Linear,bert-base-uncased,5.188790,7.000000,97.793763,26.398925,0.846716,0.000000,1.484296,1.494318
4,24,256,29009.903616,450.000000,6.529492,6.128855,8946783.200000,5.758720,0.889357,14.582152,encoder.layer.0.intermediate.dense:Linear,bert-base-uncased,5.128502,7.000000,100.000000,37.000000,0.745677,0.000000,0.000000,0.000000
5,24,256,28995.747840,342.000000,6.194738,6.469192,8027977.600000,6.005586,0.576576,16.347543,encoder.layer.0.output.dense:Linear,bert-base-uncased,5.242173,7.000000,100.000000,17.047449,1.672736,0.000000,0.000000,0.159727
6,24,256,28.311552,2.390625,5.015522,0.315138,4168206.200000,1.853969,0.074344,2.979504,pooler.dense:Linear,bert-base-uncased,5.140552,16.000000,77.437011,2.918391,1.105012,0.000000,2.168797,1.857397
7,24,256,0.073728,0.070312,5.016610,0.258492,2095743.600000,2.403729,0.023746,2.043790,pooler.activation:Tanh,bert-base-uncased,5.097471,16.000000,21.592874,0.000000,1.333933,0.000000,1.819815,NaN
8,32,256,0.000000,24.062500,5.147663,1.592306,7004202.600000,6.064460,0.256511,66.583193,embeddings.word_embeddings:Embedding,bert-base-uncased,5.538280,4.000000,97.127527,52.242581,3.240511,0.000000,1.566627,3.069650
9,32,256,0.000000,0.751953,5.028560,0.726027,4320531.800000,1.109001,0.047165,2.292423,embeddings.position_embeddings:Embedding,bert-base-uncased,5.106575,4.000000,46.018621,9.601379,0.877470,0.000000,2.171739,2.510006


In [45]:
module_info = pd.concat(infos)
module_info

,batch_size,seq_len,flops,mem_bytes,times_mean,times_std,gpu_power_mean,gpu_power_std,repeats,energy_mean,...,level_name,model_name,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std
0,24,256,47.185920,180.845703,5.062022,0.868305,6937243.600000,1.286539,6250,0.287454,...,embeddings:BertEmbeddings,bert-base-uncased,5.399172,5.000000,97.144828,70.932644,2.735842,0.000000,2.012103,1.989712
1,24,256,26721.386496,1098.000000,5.497614,3.621481,6486775.000000,2.474559,900,21.005856,...,encoder.layer.0.attention.self:BertSelfAttention,bert-base-uncased,5.521900,5.000000,97.807460,47.737061,2.210000,0.000000,1.631789,1.692584
2,24,256,7294.943232,234.000000,5.236244,2.569426,6136128.000000,2.200712,3900,4.665723,...,encoder.layer.0.attention.output:BertSelfOutput,bert-base-uncased,5.303284,11.000000,97.825806,41.293763,2.519041,0.000000,1.815847,2.114108
3,24,256,34016.329728,1332.000000,5.228161,3.331191,6118802.400000,3.721521,700,25.979729,...,encoder.layer.0.attention:BertAttention,bert-base-uncased,5.560555,14.000000,97.771183,46.154819,1.495867,0.000000,1.921325,2.266140
4,24,256,29123.149824,594.000000,5.838379,6.842611,7786088.400000,6.575166,1550,0.749523,...,encoder.layer.0.intermediate:BertIntermediate,bert-base-uncased,5.174033,16.000000,100.000000,43.000000,0.652676,0.000000,0.000000,0.000000
5,24,256,29038.215168,450.000000,5.176586,6.805338,6641509.200000,5.001363,1450,3.324903,...,encoder.layer.0.output:BertOutput,bert-base-uncased,5.645431,19.000000,97.487586,22.646707,1.459028,0.000000,1.339995,1.077452
6,24,256,92177.694720,2376.000000,5.917524,2.828442,7746734.600000,6.847845,400,17.672025,...,encoder.layer.0:BertLayer,bert-base-uncased,5.800599,22.000000,97.465546,39.871102,2.160222,0.000000,2.010766,2.634677
7,24,256,1106132.336640,28512.000000,9.177239,0.574729,11774289.400000,6.960894,50,389.685400,...,encoder:BertEncoder,bert-base-uncased,5.729913,30.000000,98.810927,40.270877,1.594745,0.000000,0.925426,2.757763
8,24,256,28.385280,2.460938,4.986758,0.541034,3537539.600000,1.063674,100950,0.131692,...,pooler:BertPooler,bert-base-uncased,5.147057,30.000000,45.805287,1.945747,1.831209,0.000000,2.468969,1.766453
9,32,256,62.914560,240.876953,5.086185,1.141099,7069030.000000,1.588011,4800,0.412581,...,embeddings:BertEmbeddings,bert-base-uncased,5.231126,5.000000,97.896782,72.437471,2.110210,0.000000,1.752465,1.743754


In [19]:
model_info = pd.concat(infos)
model_info

,batch_size,seq_len,flops,mem_bytes,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std
0,24,256,1106207.920128,28695.447266,7.120476,2.602448,9597356.600000,7.987137,274.894850,69.111278,bert-base-uncased,bert-base-uncased,5.785132,7.000000,99.060627,41.156653,1.741948,0.000000,0.892908,2.112053
1,32,256,1474943.893504,38259.595703,6.689205,3.699428,9206144.600000,5.403944,94.990800,14.196648,bert-base-uncased,bert-base-uncased,5.662072,9.000000,98.535365,42.828773,0.618296,0.000000,0.924497,1.961747
0,32,256,737213.939712,17184.125000,5.336010,4.621638,7398663.600000,5.697597,41.158200,2.719258,distilbert-base-uncased,distilbert-base-uncased,5.769084,7.000000,97.361290,38.838001,1.346249,0.000000,1.384359,1.578097
0,32,256,1474943.909888,38283.125000,6.598490,3.468873,9158014.600000,5.969359,78.406400,32.493403,roberta-base,roberta-base,5.625805,8.000000,97.693559,43.114748,1.774525,0.000000,1.807026,3.192929


In [ ]:
infos[2].dropna().sort_values('energy_std')

In [ ]:
infos[3].dropna().sort_values('energy_std')